# Henry Salgado
# Date: 10.06.24


This notebook performs a Mann-Whitney U test (since the data is not normally distributed). 

In [2]:
import pandas as pd
import numpy as np
import scipy 

instructional_data = pd.read_excel('hsi_stem_instructional.xlsx')

Q250A	Department as a place to work - All things considered, your department as a place to work
Q250B	Institution as a place to work - All things considered, your institution as a place to work

In [3]:
from scipy.stats import wilcoxon
from scipy.stats import norm


dept_satisfaction = instructional_data['q250a']  
inst_satisfaction = instructional_data['q250b']  


stat, p_value = wilcoxon(dept_satisfaction, inst_satisfaction)


print(f'Wilcoxon Test Statistic: {stat}')
print(f'p-value: {p_value}')


z = abs(norm.ppf(p_value / 2))  

# Number of non-zero differences (i.e., exclude zero differences for Wilcoxon test)
n = np.sum(dept_satisfaction != inst_satisfaction)

# Calculate the effect size r
effect_size = z / np.sqrt(n)

# Output the effect size
print(f'Effect Size (r): {effect_size}')

Wilcoxon Test Statistic: 12697.5
p-value: 1.1967084565174329e-29
Effect Size (r): 0.5778162294632012
